# Pull ESPN Ownership Data

### MSP 9.4.2017


In [41]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
# boilerplate imports

import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot




In [2]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator



### Change In Ownership

Also set up an archive.

In [3]:

Ownership = {}


for tablenum in range(0,13):

    # cycle through all positions
    r  = requests.get('http://games.espn.com/flb/addeddropped?slotCategoryId='+str(tablenum))

    data = r.text

    soup = BeautifulSoup(data)

    tables = soup.findAll('table')


    for table in tables: 

        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        #print headings


        for row in table.find_all("tr")[1:]: 

        #    sav = [td.find('a') for td in row.find_all("td")]
            sav = [td.get_text() for td in row.find_all("td")]

            w = sav.index(u'\xa0')

            #print (sav[1].split(',')[0]).strip('*')

            try:
                # block out all non-rankings
                xx = float(sav[0])

                #print sav[0:w]
                #print sav[w+1:len(sav)]

                # added
                Ownership[(sav[1].split(',')[0]).strip('*')] = float(sav[5])

                #dropped
                Ownership[(sav[w+2].split(',')[0]).strip('*')] = float(sav[w+6])

            except:
                pass

            #[u'RK', u'PLAYER, TEAM', u'POS', u'LAST', u'CUR', u'7-DAY +/-']
        

today = datetime.date.today()

f = open('ownership-archive/'+str(today)+'.txt','w')

for index,key in enumerate(Ownership.keys()):
    print >>f,key,Ownership[key]
    
f.close()

In [5]:
print l[0]

positives = ['congratulates']

print np.sum(np.array([(x in l[0]) for x in positives]))



NameError: name 'l' is not defined

In [20]:
Ownership = {}
Ownership['Jose Abreu'] = -10.

In [26]:
negatives = ['rehab','injure','disable','DL','suspension','surgery','injury','pending']

#[(x in string_in) for x in negatives]

def check_words(string_in,words):
    total = [(x in string_in) for x in words]
    return np.sum(np.array(total))
    

for index,key in enumerate(Ownership.keys()):
    if Ownership[key] < -5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print check_words(l[0],negatives),' ',
        print check_words(l[1],negatives),' ',
        print check_words(l[2],negatives)
        print ''



Jose Abreu
--------------
Most notably, how will the futures of first baseman Jose Abreu and outfielder Avisail Garcia be impacted? Both Abreu and Garcia are coming off strong seasons and have two years of contract control left. Abreu turned 31 Monday and will make $13 million in 2018. With his arbitration process pending, the ...
1   0   0



In [23]:
for index,key in enumerate(Ownership.keys()):
    if Ownership[key] > 5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print ''




In [51]:
print Ownership['Justin Verlander']

KeyError: 'Justin Verlander'

### News Alerts

Another interesting thing is grabbing Google News findings on players to see if we can determine what the narrative leading to the explosion in pickups or drops is coming from.

Good keywords:
1. Trade
2. Hot
3. Streak
4. Fire
5. Homer


Bad keywords:
1. Cold
2. Bench
3. Disabled/DL
4. Injure
5. Suspension



In [34]:
#https://stackoverflow.com/questions/39354587/scraping-google-news-with-beautifulsoup-returns-empty-results

from bs4 import BeautifulSoup
import requests
import time
from random import randint


def scrape_news_summaries(s):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    
    r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")    
    # Print the status code
    #print(r.status_code)  
    
    content = r.text
    #print content
    
    news_summaries = []
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.find_all("div", class_="gs-info")
    st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("div", {"class": "rc"})


    for st_div in st_divs:
        #print st_div
        news_summaries.append(st_div.text)
    return news_summaries




#http://www.espn.com/search/results?q=Alex+Wood
#r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")

l = scrape_news_summaries("Jose Abreu")

for n in l:
    print(n)
    print('')

Most notably, how will the futures of first baseman Jose Abreu and outfielder Avisail Garcia be impacted? Both Abreu and Garcia are coming off strong seasons and have two years of contract control left. Abreu turned 31 Monday and will make $13 million in 2018. With his arbitration process pending, the ...

CHICAGO – The numbers are just part of the picture. Jose Abreu's value to the White Sox extends far beyond the field. So while the rebuilding project continues on the South Side, Abreu remains with the only major league team he has ever known. The slugging first baseman means so much to Chicago in ...

I have been adamant and consistent this offseason. I like Jose Abreu and Avi Garcia. I like Abreu's consistency and stoicism. I like Garcia's hustle and persistence. Despite this affinity, I think they both should be traded. Abreu has two seasons of control left, and he'll be 33 in 2020 when the White Sox are ...

With the Chicago White Sox going all-in for a rebuild and dealing most o

In [14]:

def find_results(player,timescale='w'):
    r = requests.get("http://www.google.com/search?q="+player+"&tbm=nws&tbs=qdr:"+timescale)    

    content = r.text
    soup = BeautifulSoup(content, "html.parser")

    st_divs = soup.findAll("div", {"id":"resultStats"})
    print(st_divs[0].text)
    
    try:
        if st_divs[0].text.split()[0] == 'About':

            return float(st_divs[0].text.split()[1].replace(",",""))

        else:
            return float(st_divs[0].text.split()[0].replace(",",""))

    except:
            return 0.

In [4]:

HDict = {}

f = open('data/batterdict.dat')

for line in f:
    #print line
    try:
        HDict[(line.split('|')[0]).strip()] = line.split('|')[1]
    except:
        pass
    
    
print('Found {} hitters by scraping teams.'.format(len(HDict.keys())))

Found 340 hitters by scraping teams.


In [21]:
NDict = {}

for key in HDict.keys():
    
    print key,
    
    NDict[key] = find_results(key,timescale='w')



Rougned Odor About 296 results
Justin Upton About 5,680 results
Rhys Hoskins About 918 results
Kevan Smith 9 results
Ryan Braun About 2,680 results
Jose Bautista About 5,910 results
Carlos Asuaje About 259 results
Bruce Maxwell About 4,270 results
Matt Olson About 2,540 results
Stephen Piscotty About 893 results
Wilson Ramos About 1,910 results
Cameron Maybin About 908 results
Hernan Perez About 2,050 results
Adam Duvall About 975 results
Adrian Gonzalez About 4,890 results
Matt Joyce About 4,490 results
Brad Miller About 21,100 results
Daniel Robertson About 3,600 results
Dustin Pedroia About 320 results
Gary Sanchez About 13,000 results
J.T. Realmuto About 428 results
Nori Aoki 2 results
Joey Gallo About 2,400 results
Evan Gattis About 502 results
Gregor Blanco About 218 results
Ian Kinsler About 1,630 results
Chris Owings About 170 results
Christian Vazquez About 1,970 results
Kevin Pillar About 1,350 results
J.T. Riddle 8 results
Mike Zunino About 100 results
Greg Garcia About 2,63

In [29]:

for key in NDict.keys():
    
    if NDict[key] > 70000.: print key

Chris Young
Michael Taylor


In [10]:
l = scrape_news_summaries("'Trever+Story'")

for n in l:
    print(n)
    print('')

NameError: name 'scrape_news_summaries' is not defined

### Stats Alerts

We should also scrape the fangraphs gamelog page to see if there is anything obvious there.

### Crazy

It would be awesome to try and make a model to predict ownership data based on news stories.

In [14]:
r  = requests.get('http://www.espn.com/mlb/player/_/id/33095/jose-abreu')

data = r.text

soup = BeautifulSoup(data)

fantasy_table = soup.find_all('div', id='fantasy-content')

print fantasy_table#.find("tr")

#for table in tables: 

#        headings = [th.get_text() for th in table.find("tr").find_all("th")]
#        #print headings

#<div class="mod-content" id="fantasy-content">


#(row in table.find_all("tr", class_="grid_postseason"))

[<div class="mod-content" id="fantasy-content">{"playerId":5600,"percentChange":0,"averageDraftPosition":"Undrafted","percentOwned":0,"playerRaterSEASON":8.4,"fullName":"Jose Abreu","positionRank":4}</div>]
